In [1]:
from mediapipe.framework.formats.landmark_pb2 import NormalizedLandmark
empty_pivot = NormalizedLandmark()
empty_pivot.x = 0 
empty_pivot.y = 0 
empty_pivot.z = 0 
empty_pivot.visibility = 0 

def distance(pt1, pt2):
    return ((pt1.x-pt2.x)**2 + (pt1.y-pt2.y)**2 + (pt1.z-pt2.z)**2)**0.5

# Return: status, Landmarks, left vaild landmark, right valid landmark
# Valid landmark: landmarks that are detected, else empty list
# status: -1: not detected, 0: two hands, 1: left hand only, 2: right hand only
# TODO: Handle Rotation/ Flip values here later
def get_flatten_landmarks(image, holistic, use_visibility=False, scale=False):
    if image is None:
        return -1, [None]
    if(image.shape[0]==0 or image.shape[1]==0):
        return -1, [None]
    results = holistic.process(image)

    status = -1

    # Extract landmarks
    pose = results.pose_landmarks
    left_hand = results.left_hand_landmarks
    right_hand = results.right_hand_landmarks

    # Counting this to prevent the image show a tiny part of hands (20 is the max)
    left_boundary_count = 0
    right_boundary_count = 0

    if pose is not None:
        if scale:
            pivot = pose.landmark[0]
            a, b = pose.landmark[11], pose.landmark[12]
            scale_length = distance(a,b)
        else:
            pivot = dataset.empty_pivot
            scale_length = 1

        pose_row = None
        for i, landmark in enumerate(pose.landmark):
#                 if i not in dataset.keep_pose:
#                     continue # Skip Non-informative Landmarks
            landmark_x = (landmark.x-pivot.x)/scale_length
            landmark_y = (landmark.y-pivot.y)/scale_length
            landmark_z = (landmark.z-pivot.z)/scale_length
            visibility = landmark.visibility

            pose_row = landmark_x if pose_row is None else np.append(pose_row, landmark_x)
            pose_row = np.append(pose_row, landmark_y)
            pose_row = np.append(pose_row, landmark_z)
            if use_visibility:
                pose_row = np.append(pose_row, visibility)
    else:
        status = -1
        # Invalid Image
        return status, [None]

    if left_hand is not None:
        if scale:
            pivot_left = left_hand.landmark[0]
            a, b = left_hand.landmark[5], left_hand.landmark[0]
            scale_length_left = distance(a,b)

        left_row = None
        for landmark in left_hand.landmark:
            landmark_x_relative_hand = (landmark.x-pivot_left.x)/scale_length_left
            landmark_y_relative_hand = (landmark.y-pivot_left.y)/scale_length_left
            landmark_z_relative_hand = (landmark.z-pivot_left.z)/scale_length_left

            landmark_x = (landmark.x-pivot.x)/scale_length
            landmark_y = (landmark.y-pivot.y)/scale_length
            landmark_z = (landmark.z-pivot.z)/scale_length

            left_row = landmark_x if left_row is None else np.append(left_row, landmark_x)
            left_row = np.append(left_row, landmark_y)
            left_row = np.append(left_row, landmark_z)

            left_row = np.append(left_row, landmark_x_relative_hand)
            left_row = np.append(left_row, landmark_y_relative_hand)
            left_row = np.append(left_row, landmark_z_relative_hand)

            if (not (landmark.x > 0  and landmark.x < 1)) or not (landmark.y > 0  and landmark.y < 1):
                left_boundary_count+=1

    if right_hand is not None:
        if scale:
            pivot_right = right_hand.landmark[0]
            a, b = right_hand.landmark[5], right_hand.landmark[0]
            scale_length_right = distance(a,b)

        right_row = None
        for landmark in right_hand.landmark:
            landmark_x_relative_hand = (landmark.x-pivot_right.x)/scale_length_right
            landmark_y_relative_hand = (landmark.y-pivot_right.y)/scale_length_right
            landmark_z_relative_hand = (landmark.z-pivot_right.z)/scale_length_right

            landmark_x = (landmark.x-pivot.x)/scale_length
            landmark_y = (landmark.y-pivot.y)/scale_length
            landmark_z = (landmark.z-pivot.z)/scale_length

            right_row = landmark_x if right_row is None else np.append(right_row, landmark_x)
            right_row = np.append(right_row, landmark_y)
            right_row = np.append(right_row, landmark_z)

            right_row = np.append(right_row, landmark_x_relative_hand)
            right_row = np.append(right_row, landmark_y_relative_hand)
            right_row = np.append(right_row, landmark_z_relative_hand)

            if (not (landmark.x > 0  and landmark.x < 1)) or not (landmark.y > 0  and landmark.y < 1):
                right_boundary_count+=1

    # Two Hands Detected, but could be only showing a very tiny part
    if left_hand is not None and right_hand is not None:
        result = np.concatenate([pose_row,left_row,right_row])
        result = result.reshape((1,result.shape[0]))
        #print('both',result.shape)
        status = 0
        if left_boundary_count > 10: # Left Hand Invalid
            status = 2
        if right_boundary_count > 10: # Right Hand Invalid
            status = 1
        if left_boundary_count > 10 and right_boundary_count > 10: # Two Hands Are Invalid
            status = -1
    else:
        # x->0.5 = middle, y->1 = bottom

        #Single Hand Model
        if left_hand is not None:
            empty_arr = np.tile([0,0,0], 21*2)
            result = np.concatenate([pose_row,left_row,empty_arr])
            result = result.reshape((1,result.shape[0]))
            #print('left',result.shape)
            status = 1
        elif right_hand is not None:
            empty_arr = np.tile([0,0,0], 21*2)
            result = np.concatenate([pose_row, empty_arr, right_row])
            result = result.reshape((1,result.shape[0]))
            #print('right',result.shape)
            status = 2
        else:
            result = [None]
            status = -1
    return status, result

        
class frame_buffer:
    import numpy as np
    # Shape should be a 2d tuple
    def __init__(self, shape):
        self.shape = shape
        self.buffer = np.zeros(shape)
        self.pointer = 0
        self.item_count = 0
    
    def append(self, row):
        #shape[0] : number of rows
        self.buffer[self.pointer] = row
        if self.item_count < self.shape[0]:
            self.item_count += 1
        if self.pointer < self.shape[0]-1:
            self.pointer += 1
        else:
            # Reset counting from zero
            self.pointer = 0
            
    def generate(self):
        result = np.zeros(self.shape)
        counter = 0
        for i in range(self.pointer, self.shape[0]):
            result[counter] = self.buffer[i]
            counter += 1
        for j in range(0, self.pointer):
            result[counter] = self.buffer[j]
            counter += 1
        result = result.reshape(1,self.shape[0],self.shape[1])
        return result
    
    def isFill(self):
        return self.item_count == self.shape[0]
    
    

C:\Users\homan\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import pickle
import tensorflow as tf
import cv2
import mediapipe as mp
import traceback
import numpy as np
from sklearn.preprocessing import LabelEncoder




In [3]:
def selectModel(idx):
    models = [
        {'path':'models_to_test\model_final.tflite', 'visibility':False,'scale':True, 'label':'models_to_test\OnehotEncoder_final.pkl'}
    ]
    model = tf.lite.Interpreter(model_path=models[idx]['path'])
    label = models[idx].get("label")
    return model, models[idx]['visibility'], models[idx]['scale'], label

In [6]:
model, visibility, scale, label= selectModel(0)
model.allocate_tensors()
input_details = model.get_input_details()
output_details = model.get_output_details()

if label is None:
    file = open(f'models_to_test\OnehotEncoder.pkl','rb')
    encoder = pickle.load(file)
else:
    file = open(label,'rb')
    encoder = pickle.load(file)
    
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Test Model
#cap = cv2.VideoCapture('videos_reconstruct/test/family/0/20988.mp4')
cap = cv2.VideoCapture(0)
#set the width and height, and UNSUCCESSFULLY set the exposure time
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1280)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    buffer = frame_buffer(shape=(20,351))
        
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        #frame = frame[:,50:frame.shape[0]-50]

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = image[:,100:-100]
        image = image.copy()
        image.flags.writeable = False        
        
        # Flip Output
        frame = cv2.flip(frame,1)
        results = holistic.process(image)

        # Concate rows
        row = get_flatten_landmarks(image, holistic, visibility, scale)
        #print(row[1])
#         if row[0] != -1:
#             print(row[1].flatten())
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Export coordinates
        try:
            
            # 1. Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )

            # 2. Right hand
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                     )

            # 3. Left Hand
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                     )
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))


            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            
            if row[0] != -1:
                buffer.append(row[1])
                if buffer.isFill():
                    to_predict = buffer.generate()
                    to_predict = np.float32(to_predict)
                    model.set_tensor(input_details[0]['index'], to_predict)
                    model.invoke()
                    predict_result = model.get_tensor(output_details[0]['index'])
                    
                    predict_label = np.argmax(predict_result, axis=None, out=None)
                    body_language_prob = predict_result[0, predict_label]
                    body_language_class = encoder.inverse_transform(predict_result)[0]
                    
                    cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
                    
                    # Display Class
                    cv2.putText(image, 'CLASS'
                                , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(body_language_class)
                                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.putText(image, str(body_language_class), coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    
                    # Display Probability
                    cv2.putText(image, 'PROB'
                                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(round(body_language_prob,2))
                                 , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except Exception as e:
            # print(traceback.format_exc())
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\homan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
with open('labels.txt', 'w') as f:
    for i in range(84):
        onehot_empty = np.zeros((1,84))
        onehot_empty[0][i] = 1
        k = encoder.inverse_transform(onehot_empty)[0][0]
        for i in range(0,10):
            k = k.replace(f"_{i}","")
        f.write("%s\n" % k)
